In [10]:
import requests
# get alpaca api keys using this guide: https://alpaca.markets/docs/market-data/getting-started/#creating-an-alpaca-account-and-finding-your-api-keys
from secrets_config import api_key_id, api_secret_key 
import pandas as pd

In [11]:
import datetime as dt 

def generate_datetime_ranges(
        start_date:str=None, 
        end_date:str=None, 
    )->list:
    """ 
    Generates a range of datetime ranges. 
    - start_date: provide a str with the format "yyyy-mm-dd"
    - end_date: provide a str with the format "yyyy-mm-dd" 
    Usage example: 
    - generate_datetime_ranges(start_date="2020-01-01", end_date="2022-01-02")
        returns: 
            [
                'start_time': '2020-01-01T00:00:00.00Z', 'end_time': '2020-01-02T00:00:00.00Z'}, 
                {'start_time': '2020-01-02T00:00:00.00Z', 'end_time': '2020-01-03T00:00:00.00Z'}
            ]
    """

    date_range = []
    if start_date is not None and end_date is not None: 
        dte_start_date = dt.datetime.strptime(start_date, "%Y-%m-%d")
        dte_end_date = dt.datetime.strptime(end_date, "%Y-%m-%d")
        date_range = [
            {
                "start_time": (dte_start_date + dt.timedelta(days=i)).strftime("%Y-%m-%dT%H:%M:%S.00Z"),
                "end_time": (dte_start_date + dt.timedelta(days=i) + dt.timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%S.00Z"),
            }
        for i in range((dte_end_date - dte_start_date).days)]
    else: 
        raise Exception("The parameters passed in results in no action being performed.")

    return date_range  

In [12]:
# docs: https://alpaca.markets/docs/api-references/market-data-api/stock-pricing-data/historical/

"""
request data with the following parameters: 
- stock: tesla 
- start date: 01/01/2020
- end date: 02/01/2020
"""
stock_ticker = "tsla" # tlsa maps to tesla
base_url = f"https://data.alpaca.markets/v2/stocks/{stock_ticker}/trades"
start_date = "2020-01-01"
end_date = "2020-01-05"
response_data = []

for date in generate_datetime_ranges(start_date=start_date, end_date=end_date):
    start_time = date.get("start_time")
    end_time = date.get("end_time")

    print(start_time, end_time)

    """
    to authenticate to the api, you will need to use the APCA-API-KEY-ID and APCA-API-SECRET-KEY fields. 
    for example:
        headers = {
            "APCA-API-KEY-ID": "<your_api_key_id>",
            "APCA-API-SECRET-KEY": "<your_api_secret_key>"
        }   

    and in the request.get() method, you will have to use: 
        requests.get(url=url, params=params, headers=headers)
    """
    
    params = {
        "start": start_time,
        "end": end_time
    }

    # auth example: https://alpaca.markets/docs/api-references/trading-api/
    headers = {
        "APCA-API-KEY-ID": api_key_id,
        "APCA-API-SECRET-KEY": api_secret_key
    }
    response = requests.get(base_url, params=params, headers=headers)
    if response.json().get("trades") is not None: 
        response_data.extend(response.json().get("trades"))

2020-01-01T00:00:00.00Z 2020-01-02T00:00:00.00Z
2020-01-02T00:00:00.00Z 2020-01-03T00:00:00.00Z
2020-01-03T00:00:00.00Z 2020-01-04T00:00:00.00Z
2020-01-04T00:00:00.00Z 2020-01-05T00:00:00.00Z


In [13]:
# read json data to a dataframe 
df_quotes = pd.json_normalize(data=response_data, meta=["symbol"])
# rename columns to more meaningful names
df_quotes_renamed = df_quotes.rename(columns={
    "t": "timestamp",
    "x": "exchange",
    "p": "price",
    "s": "size",
})

In [14]:
# select only 'timestamp', 'exchange', 'price', 'size'
df_quotes_selected = df_quotes_renamed[['timestamp', 'exchange', 'price', 'size']]

In [15]:
df_exchange_codes = pd.read_csv("data/exchange_codes.csv")

In [16]:
df_exchange = pd.merge(left=df_quotes_selected, right=df_exchange_codes, left_on="exchange", right_on="exchange_code").drop(columns=["exchange_code", "exchange"]).rename(columns={"exchange_name": "exchange"})
df_exchange.head()

,timestamp,price,size,exchange
0,2020-01-01T00:00:20.4997Z,418.93,60,Cboe EDGX
1,2020-01-01T00:00:38.5731Z,418.70,10,Cboe EDGX
2,2020-01-01T00:09:35.5596Z,418.75,20,Cboe EDGX
3,2020-01-01T00:10:13.0418Z,418.75,115,Cboe EDGX
4,2020-01-01T00:10:28.0684Z,418.75,200,Cboe EDGX


In [17]:
# remove duplicates by doing a group by on the keys: timestamp and exchange
# get the mean of price, and sum of size
df_ask_bid_exchange_de_dup = df_exchange.groupby(["timestamp", "exchange"]).agg({
    "price": "mean",
    "size": "sum",
}).reset_index()
df_ask_bid_exchange_de_dup.head()

,timestamp,exchange,price,size
0,2020-01-01T00:00:20.4997Z,Cboe EDGX,418.93,60
1,2020-01-01T00:00:38.5731Z,Cboe EDGX,418.70,10
2,2020-01-01T00:00:50.8222Z,NASDAQ Int,418.74,5
3,2020-01-01T00:06:12.9975Z,FINRA ADF,418.75,8
4,2020-01-01T00:09:35.5596Z,Cboe EDGX,418.75,20


Create new file with new timestamp of the ingestion start and end times

In [18]:
date_ranges = generate_datetime_ranges(start_date=start_date, end_date=end_date)
start_time = date_ranges[0]["start_time"]
end_time = date_ranges[-1]["end_time"]

df_ask_bid_exchange_de_dup.to_parquet(f"data/exchange_data_{start_time}_{end_time}.parquet", index=False)

Upsert data into postgres database table using timestamp as the primary key

In [19]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from secrets_config import db_user, db_password, db_server_name, db_database_name
from sqlalchemy.schema import CreateTable 

In [20]:
# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)

engine = create_engine(connection_url)


In [21]:
df_ask_bid_exchange_de_dup.columns

Index(['timestamp', 'exchange', 'price', 'size'], dtype='object')

In [22]:
df_ask_bid_exchange_de_dup

,timestamp,exchange,price,size
0,2020-01-01T00:00:20.4997Z,Cboe EDGX,418.930000,60
1,2020-01-01T00:00:38.5731Z,Cboe EDGX,418.700000,10
2,2020-01-01T00:00:50.8222Z,NASDAQ Int,418.740000,5
3,2020-01-01T00:06:12.9975Z,FINRA ADF,418.750000,8
4,2020-01-01T00:09:35.5596Z,Cboe EDGX,418.750000,20
...,...,...,...,...
1609,2020-01-04T00:59:13.4401Z,NASDAQ Int,442.000000,39
1610,2020-01-04T00:59:13.784Z,Cboe EDGX,442.200000,39
1611,2020-01-04T00:59:33.685Z,Cboe EDGX,442.066667,6
1612,2020-01-04T00:59:45.5109Z,FINRA ADF,441.910000,150


In [23]:
meta = MetaData()
stock_price_tesla_table = Table(
    "stock_price_tesla", meta, 
    Column("timestamp", String, primary_key=True),
    Column("exchange", String, primary_key=True),
    Column("price", Float),
    Column("size", Integer)
)
meta.create_all(engine) # creates table if it does not exist 


In [24]:
insert_statement = postgresql.insert(stock_price_tesla_table).values(df_ask_bid_exchange_de_dup.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['timestamp', 'exchange'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['timestamp','exchange']})
engine.execute(upsert_statement)